In [ ]:
from flask import Flask, render_template_string
import psycopg2
app = Flask(__name__)



@app.route('/static/<path:filename>')
def serve_static(filename):
    return app.send_static_file(filename)


@app.route('/')
def home():
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        database="Candidate_List",
        user="postgres",
        password="Adduvaith789",
        host="localhost",
        port="5432"
    )

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # For the Marital Status - Pie chart
    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Married';")
    rows1 = cursor.fetchone()

    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Single';")
    rows2 = cursor.fetchone()

    count1 = rows1[0]
    count2 = rows2[0]

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='male';")
    rows1a = cursor.fetchone()

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='female';")
    rows2a = cursor.fetchone()

    count1a = rows1a[0]
    count2a = rows2a[0]
    
    
    #line chart of Grades
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Permanent';")
    avg = cursor.fetchone()
    avgl1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Contract';")
    avg = cursor.fetchone()
    avgl2 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Consultant';")
    avg = cursor.fetchone()
    avgl3 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Probationary';")
    avg = cursor.fetchone()
    avgw1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Freelancers';")
    avg = cursor.fetchone()
    avgw2 = avg[0]
    
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Interns';")
    avg = cursor.fetchone()
    avgw3 = avg[0]
    
    cursor.execute("SELECT grade, enter_ctc, CAST(REGEXP_REPLACE(level_, '[^\d]', '', 'g') AS INT) AS levels FROM list;")

    # Fetch all the rows from the result set
    rows = cursor.fetchall()
    
    
    
    cursor.execute("SELECT designation, COUNT(designation) AS Dcount FROM list GROUP BY designation;")
    rows4 = cursor.fetchall()
    
    
    cursor.execute("SELECT COUNT(department) AS Dcount, SUM(enter_ctc) FROM list GROUP BY department;")
    rows5 = cursor.fetchall()
    
    

    # Close the cursor and database connection
    cursor.close()
    conn.close()
    
    data6 = [['Grade', 'Employee CTC', 'Level']]
    data7 = [['Designation', 'Count']]
    data8 = [['Count of Departments', 'CTC']]

    # Append rows from the database to the data6 variable
    for row in rows:
        data6.append(list(row))
    
    for row in rows4:
        data7.append(list(row))
    
    
    for row in rows5:
        data8.append(list(row))


    
    html_code = '''
    <html>
    <head>
      <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
      <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
      <script type="text/javascript">
        google.charts.load("current", {packages:["corechart"]});
        google.charts.setOnLoadCallback(drawCharts);

        function drawCharts() {
          var data1 = google.visualization.arrayToDataTable([
            ['Marital Status', 'Count'],
            ['Married', {{ count1 }}],
            ['Unmarried', {{ count2 }}]
          ]);

          var options1 = {
            title: 'Marital Status',
            is3D: true,
          };

          var chart1 = new google.visualization.PieChart(document.getElementById('donutchart1'));
          chart1.draw(data1, options1);










          var data2 = google.visualization.arrayToDataTable([
            ['Gender', 'Value', { role: 'style' }],
            ['Male', {{ count1a }}, 'stroke-color: #871B47; stroke-opacity: 0.6; stroke-width: 2; fill-color: #BC5679; fill-opacity: 0.2'],
            ['Female', {{ count2a }}, 'stroke-color: #871B47; stroke-opacity: 0.6; stroke-width: 2; fill-color: #BC5679; fill-opacity: 0.2']
          ]);

          var options2 = {
            title: 'Gender Ratio',
            legend: 'none'
          };

          var chart2 = new google.visualization.BarChart(document.getElementById('chart_div1'));
          chart2.draw(data2, options2);

          var data3 = google.visualization.arrayToDataTable([
            ['Employee Category', 'Part Time', 'Full Time'],
            ['', {{ avgl1 }}, {{ avgw1 }}],
            ['', {{ avgw2 }}, {{ avgl2 }}],
            ['', {{ avgl3 }}, {{ avgw3 }}]
          ]);

          var options3 = {
            title: 'Employee Category - Part time VS Full time',
            curveType: 'function',
            legend: { position: 'bottom' }
          };

          var chart3 = new google.visualization.LineChart(document.getElementById('curve_chart'));
          chart3.draw(data3, options3);
          
          var data4 = google.visualization.arrayToDataTable({{ data6|safe }});

            var options4 = {
              title: 'Grade Levels',
              hAxis: { title: 'Employee CTC' },
              vAxis: { title: 'Level' },
              bubble: {
                textStyle: { fontSize: 11 },
                stroke: '#000000', // Set the stroke color for the bubbles
                fill: '#ff0000', // Set the fill color for the bubbles
                opacity: 0.6, // Set the opacity for the bubbles (0.0 - 1.0)
                sizes: [10, 30], // Set the range of sizes for the bubbles
              },
        };

            var chart4 = new google.visualization.BubbleChart(document.getElementById('chart'));
            chart4.draw(data4, options4);
            
            
            
            
            
          
          var data4 = google.visualization.arrayToDataTable({{ data7|safe }});

          var options4 = {
            title: 'Designation and their Count',
            vAxis: { title: 'Count' },
            legend: { position: 'none' }
          };

          var chart4 = new google.visualization.ColumnChart(document.getElementById('column_chart'));
          chart4.draw(data4, options4);
          
          
          
          
          
          
          
          
          
          
          
          var data = new google.visualization.DataTable();
          data.addColumn('number', 'X');
          data.addColumn('number', 'Y');

          // Populate the data rows from the fetched data
          var rows = [
    '''

    for row in rows5:
        html_code += "        [{}],\n".format(','.join(str(value) for value in row))

    html_code += '''
    ];

          data.addRows(rows);

          var options = {
              title: 'Dept. Count vs CTCs',
              hAxis: {title: 'Dept'},
              vAxis: {title: 'CTC'},
              legend: 'none'
          };

          var chart = new google.visualization.ScatterChart(document.getElementById('scatter'));
          chart.draw(data, options);
          
            
            
        }

          $(document).ready(function() {
          $('#home').click(function() {
            window.location.href = "{{ url_for('home') }}";
          });
          $('#graph1').click(function() {
            window.location.href = "{{ url_for('graph1') }}";
          });
          $('#graph2').click(function() {
            window.location.href = "{{ url_for('graph2') }}";
          });
          $('#graph3').click(function() {
            window.location.href = "{{ url_for('graph3') }}";
          });
          $('#graph4').click(function() {
            window.location.href = "{{ url_for('graph4') }}";
          });
          $('#graph5').click(function() {
            window.location.href = "{{ url_for('graph5') }}";
          });
          $('#graph6').click(function() {
            window.location.href = "{{ url_for('graph6') }}";
          });
        });
      </script>
      <link rel="stylesheet" type="text/css" href="/static/styling.css">
    </head>
    <body>
    <ul class="graph-menu">
        <li><button id="home">Home Page</button></li>
        <li><button id="graph1">Martital Status</button></li>
        <li><button id="graph2">Gender Ratio</button></li>
        <li><button id="graph3">Part Time vs Full time</button></li>
        <li><button id="graph4">Grade vs Level View</button></li>
        <li><button id="graph5">Designation and its Count</button></li>
        <li><button id="graph6">Dept. VS CTC</button></li>
    </ul>
    <div class="container">
        <h1 class="chart-title">Chart Dashboard</h1>
        <div class="charts-container">
            <div id="donutchart1" class="chart"></div>
            <div id="chart_div1" class="chart"></div>
            <div id="curve_chart" class="chart"></div>
            <div id="chart" class="chart"></div>
            <div id="column_chart" class="chart"></div>
            <div id="scatter" class="chart"></div>  
        </div>
    </div>
    </body>
    </html>
    '''

    return render_template_string(html_code, data8 = data8, data7 = data7, count1=count1, count2=count2, count1a=count1a, count2a=count2a, avgl1=avgl1, avgl2=avgl2, avgl3=avgl3, avgw1=avgw1, avgw2=avgw2, avgw3=avgw3, data6=data6)







@app.route('/graph1')
def graph1():
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        database="Candidate_List",
        user="postgres",
        password="Adduvaith789",
        host="localhost",
        port="5432"
    )

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # For the Marital Status - Pie chart
    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Married';")
    rows1 = cursor.fetchone()

    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Single';")
    rows2 = cursor.fetchone()

    count1 = rows1[0]
    count2 = rows2[0]

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='male';")
    rows1a = cursor.fetchone()

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='female';")
    rows2a = cursor.fetchone()

    count1a = rows1a[0]
    count2a = rows2a[0]
    
    
    #line chart of Grades
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Permanent';")
    avg = cursor.fetchone()
    avgl1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Contract';")
    avg = cursor.fetchone()
    avgl2 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Consultant';")
    avg = cursor.fetchone()
    avgl3 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Probationary';")
    avg = cursor.fetchone()
    avgw1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Freelancers';")
    avg = cursor.fetchone()
    avgw2 = avg[0]
    
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Interns';")
    avg = cursor.fetchone()
    avgw3 = avg[0]
    
    cursor.execute("SELECT grade, enter_ctc, CAST(REGEXP_REPLACE(level_, '[^\d]', '', 'g') AS INT) AS levels FROM list;")

    # Fetch all the rows from the result set
    rows = cursor.fetchall()
    
    
    
    cursor.execute("SELECT designation, COUNT(designation) AS Dcount FROM list GROUP BY designation;")
    rows4 = cursor.fetchall()
    
    
    cursor.execute("SELECT COUNT(department) AS Dcount, SUM(enter_ctc) FROM list GROUP BY department;")
    rows5 = cursor.fetchall()
    
    

    # Close the cursor and database connection
    cursor.close()
    conn.close()
    
    data6 = [['Grade', 'Employee CTC', 'Level']]
    data7 = [['Designation', 'Count']]
    data8 = [['Count of Departments', 'CTC']]

    # Append rows from the database to the data6 variable
    for row in rows:
        data6.append(list(row))
    
    for row in rows4:
        data7.append(list(row))
    
    
    for row in rows5:
        data8.append(list(row))


    
    html_code = '''
    <html>
    <head>
      <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
      <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
      <script type="text/javascript">
        google.charts.load("current", {packages:["corechart"]});
        google.charts.setOnLoadCallback(drawCharts);

        function drawCharts() {
          var data1 = google.visualization.arrayToDataTable([
            ['Marital Status', 'Count'],
            ['Married', {{ count1 }}],
            ['Unmarried', {{ count2 }}]
          ]);

          var options1 = {
            title: 'Marital Status',
            is3D: true,
          };

          var chart1 = new google.visualization.PieChart(document.getElementById('donutchart1'));
          chart1.draw(data1, options1);
    }
    

    
          $(document).ready(function() {
          $('#home').click(function() {
            window.location.href = "{{ url_for('home') }}";
          });
          $('#graph1').click(function() {
            window.location.href = "{{ url_for('graph1') }}";
          });
          $('#graph2').click(function() {
            window.location.href = "{{ url_for('graph2') }}";
          });
          $('#graph3').click(function() {
            window.location.href = "{{ url_for('graph3') }}";
          });
          $('#graph4').click(function() {
            window.location.href = "{{ url_for('graph4') }}";
          });
          $('#graph5').click(function() {
            window.location.href = "{{ url_for('graph5') }}";
          });
          $('#graph6').click(function() {
            window.location.href = "{{ url_for('graph6') }}";
          });
        });
      </script>
      <link rel="stylesheet" type="text/css" href="/static/styling.css">
    </head>
    <body>
    <ul class="graph-menu">
        <li><button id="home">Home Page</button></li>
        <li><button id="graph1">Martital Status</button></li>
        <li><button id="graph2">Gender Ratio</button></li>
        <li><button id="graph3">Part Time vs Full time</button></li>
        <li><button id="graph4">Grade vs Level View</button></li>
        <li><button id="graph5">Designation and its Count</button></li>
        <li><button id="graph6">Dept. VS CTC</button></li>
    </ul>
    <div class="container">
        <h1 class="chart-title">Martital Status</h1>
        <div class="charts-container">
            <div id="donutchart1" class="chart"></div> 
        </div>
    </div>
    </body>
    </html>
    '''

    return render_template_string(html_code, data8 = data8, data7 = data7, count1=count1, count2=count2, count1a=count1a, count2a=count2a, avgl1=avgl1, avgl2=avgl2, avgl3=avgl3, avgw1=avgw1, avgw2=avgw2, avgw3=avgw3, data6=data6)













@app.route('/graph2')
def graph2():
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        database="Candidate_List",
        user="postgres",
        password="Adduvaith789",
        host="localhost",
        port="5432"
    )

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # For the Marital Status - Pie chart
    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Married';")
    rows1 = cursor.fetchone()

    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Single';")
    rows2 = cursor.fetchone()

    count1 = rows1[0]
    count2 = rows2[0]

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='male';")
    rows1a = cursor.fetchone()

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='female';")
    rows2a = cursor.fetchone()

    count1a = rows1a[0]
    count2a = rows2a[0]
    
    
    #line chart of Grades
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Permanent';")
    avg = cursor.fetchone()
    avgl1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Contract';")
    avg = cursor.fetchone()
    avgl2 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Consultant';")
    avg = cursor.fetchone()
    avgl3 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Probationary';")
    avg = cursor.fetchone()
    avgw1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Freelancers';")
    avg = cursor.fetchone()
    avgw2 = avg[0]
    
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Interns';")
    avg = cursor.fetchone()
    avgw3 = avg[0]
    
    cursor.execute("SELECT grade, enter_ctc, CAST(REGEXP_REPLACE(level_, '[^\d]', '', 'g') AS INT) AS levels FROM list;")

    # Fetch all the rows from the result set
    rows = cursor.fetchall()
    
    
    
    cursor.execute("SELECT designation, COUNT(designation) AS Dcount FROM list GROUP BY designation;")
    rows4 = cursor.fetchall()
    
    
    cursor.execute("SELECT COUNT(department) AS Dcount, SUM(enter_ctc) FROM list GROUP BY department;")
    rows5 = cursor.fetchall()
    
    

    # Close the cursor and database connection
    cursor.close()
    conn.close()
    
    data6 = [['Grade', 'Employee CTC', 'Level']]
    data7 = [['Designation', 'Count']]
    data8 = [['Count of Departments', 'CTC']]

    # Append rows from the database to the data6 variable
    for row in rows:
        data6.append(list(row))
    
    for row in rows4:
        data7.append(list(row))
    
    
    for row in rows5:
        data8.append(list(row))


    
    html_code = '''
    <html>
    <head>
      <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
      <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
      <script type="text/javascript">
        google.charts.load("current", {packages:["corechart"]});
        google.charts.setOnLoadCallback(drawCharts);

        function drawCharts() {
          var data2 = google.visualization.arrayToDataTable([
            ['Gender', 'Value', { role: 'style' }],
            ['Male', {{ count1a }}, 'stroke-color: #871B47; stroke-opacity: 0.6; stroke-width: 2; fill-color: #BC5679; fill-opacity: 0.2'],
            ['Female', {{ count2a }}, 'stroke-color: #871B47; stroke-opacity: 0.6; stroke-width: 2; fill-color: #BC5679; fill-opacity: 0.2']
          ]);

          var options2 = {
            title: 'Gender Ratio',
            legend: 'none'
          };

          var chart2 = new google.visualization.BarChart(document.getElementById('chart_div1'));
          chart2.draw(data2, options2);
    }
    

    
          $(document).ready(function() {
          $('#home').click(function() {
            window.location.href = "{{ url_for('home') }}";
          });
          $('#graph1').click(function() {
            window.location.href = "{{ url_for('graph1') }}";
          });
          $('#graph2').click(function() {
            window.location.href = "{{ url_for('graph2') }}";
          });
          $('#graph3').click(function() {
            window.location.href = "{{ url_for('graph3') }}";
          });
          $('#graph4').click(function() {
            window.location.href = "{{ url_for('graph4') }}";
          });
          $('#graph5').click(function() {
            window.location.href = "{{ url_for('graph5') }}";
          });
          $('#graph6').click(function() {
            window.location.href = "{{ url_for('graph6') }}";
          });
        });
      </script>
      <link rel="stylesheet" type="text/css" href="/static/styling.css">
    </head>
    <body>
    <ul class="graph-menu">
        <li><button id="home">Home Page</button></li>
        <li><button id="graph1">Martital Status</button></li>
        <li><button id="graph2">Gender Ratio</button></li>
        <li><button id="graph3">Part Time vs Full time</button></li>
        <li><button id="graph4">Grade vs Level View</button></li>
        <li><button id="graph5">Designation and its Count</button></li>
        <li><button id="graph6">Dept. VS CTC</button></li>
    </ul>
    <div class="container">
        <h1 class="chart-title">Gender Ratio</h1>
        <div class="charts-container">
            <div id="chart_div1" class="chart"></div> 
        </div>
    </div>
    </body>
    </html>
    '''

    return render_template_string(html_code, data8 = data8, data7 = data7, count1=count1, count2=count2, count1a=count1a, count2a=count2a, avgl1=avgl1, avgl2=avgl2, avgl3=avgl3, avgw1=avgw1, avgw2=avgw2, avgw3=avgw3, data6=data6)








@app.route('/graph3')
def graph3():
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        database="Candidate_List",
        user="postgres",
        password="Adduvaith789",
        host="localhost",
        port="5432"
    )

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # For the Marital Status - Pie chart
    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Married';")
    rows1 = cursor.fetchone()

    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Single';")
    rows2 = cursor.fetchone()

    count1 = rows1[0]
    count2 = rows2[0]

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='male';")
    rows1a = cursor.fetchone()

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='female';")
    rows2a = cursor.fetchone()

    count1a = rows1a[0]
    count2a = rows2a[0]
    
    
    #line chart of Grades
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Permanent';")
    avg = cursor.fetchone()
    avgl1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Contract';")
    avg = cursor.fetchone()
    avgl2 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Consultant';")
    avg = cursor.fetchone()
    avgl3 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Probationary';")
    avg = cursor.fetchone()
    avgw1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Freelancers';")
    avg = cursor.fetchone()
    avgw2 = avg[0]
    
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Interns';")
    avg = cursor.fetchone()
    avgw3 = avg[0]
    
    cursor.execute("SELECT grade, enter_ctc, CAST(REGEXP_REPLACE(level_, '[^\d]', '', 'g') AS INT) AS levels FROM list;")

    # Fetch all the rows from the result set
    rows = cursor.fetchall()
    
    
    
    cursor.execute("SELECT designation, COUNT(designation) AS Dcount FROM list GROUP BY designation;")
    rows4 = cursor.fetchall()
    
    
    cursor.execute("SELECT COUNT(department) AS Dcount, SUM(enter_ctc) FROM list GROUP BY department;")
    rows5 = cursor.fetchall()
    
    

    # Close the cursor and database connection
    cursor.close()
    conn.close()
    
    data6 = [['Grade', 'Employee CTC', 'Level']]
    data7 = [['Designation', 'Count']]
    data8 = [['Count of Departments', 'CTC']]

    # Append rows from the database to the data6 variable
    for row in rows:
        data6.append(list(row))
    
    for row in rows4:
        data7.append(list(row))
    
    
    for row in rows5:
        data8.append(list(row))


    
    html_code = '''
    <html>
    <head>
      <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
      <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
      <script type="text/javascript">
        google.charts.load("current", {packages:["corechart"]});
        google.charts.setOnLoadCallback(drawCharts);

        function drawCharts() {
          var data3 = google.visualization.arrayToDataTable([
            ['Employee Category', 'Part Time', 'Full Time'],
            ['', {{ avgl1 }}, {{ avgw1 }}],
            ['', {{ avgw2 }}, {{ avgl2 }}],
            ['', {{ avgl3 }}, {{ avgw3 }}]
          ]);

          var options3 = {
            title: 'Employee Category - Part time VS Full time',
            curveType: 'function',
            legend: { position: 'bottom' }
          };

          var chart3 = new google.visualization.LineChart(document.getElementById('curve_chart'));
          chart3.draw(data3, options3);
    }
    

    
          $(document).ready(function() {
          $('#home').click(function() {
            window.location.href = "{{ url_for('home') }}";
          });
          $('#graph1').click(function() {
            window.location.href = "{{ url_for('graph1') }}";
          });
          $('#graph2').click(function() {
            window.location.href = "{{ url_for('graph2') }}";
          });
          $('#graph3').click(function() {
            window.location.href = "{{ url_for('graph3') }}";
          });
          $('#graph4').click(function() {
            window.location.href = "{{ url_for('graph4') }}";
          });
          $('#graph5').click(function() {
            window.location.href = "{{ url_for('graph5') }}";
          });
          $('#graph6').click(function() {
            window.location.href = "{{ url_for('graph6') }}";
          });
        });
      </script>
      <link rel="stylesheet" type="text/css" href="/static/styling.css">
    </head>
    <body>
    <ul class="graph-menu">
        <li><button id="home">Home Page</button></li>
        <li><button id="graph1">Martital Status</button></li>
        <li><button id="graph2">Gender Ratio</button></li>
        <li><button id="graph3">Part Time vs Full time</button></li>
        <li><button id="graph4">Grade vs Level View</button></li>
        <li><button id="graph5">Designation and its Count</button></li>
        <li><button id="graph6">Dept. VS CTC</button></li>
    </ul>
    <div class="container">
        <h1 class="chart-title">Part Time vs Full time</h1>
        <div class="charts-container">
            <div id="curve_chart" class="chart"></div> 
        </div>
    </div>
    </body>
    </html>
    '''

    return render_template_string(html_code, data8 = data8, data7 = data7, count1=count1, count2=count2, count1a=count1a, count2a=count2a, avgl1=avgl1, avgl2=avgl2, avgl3=avgl3, avgw1=avgw1, avgw2=avgw2, avgw3=avgw3, data6=data6)





@app.route('/graph4')
def graph4():
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        database="Candidate_List",
        user="postgres",
        password="Adduvaith789",
        host="localhost",
        port="5432"
    )

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # For the Marital Status - Pie chart
    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Married';")
    rows1 = cursor.fetchone()

    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Single';")
    rows2 = cursor.fetchone()

    count1 = rows1[0]
    count2 = rows2[0]

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='male';")
    rows1a = cursor.fetchone()

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='female';")
    rows2a = cursor.fetchone()

    count1a = rows1a[0]
    count2a = rows2a[0]
    
    
    #line chart of Grades
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Permanent';")
    avg = cursor.fetchone()
    avgl1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Contract';")
    avg = cursor.fetchone()
    avgl2 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Consultant';")
    avg = cursor.fetchone()
    avgl3 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Probationary';")
    avg = cursor.fetchone()
    avgw1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Freelancers';")
    avg = cursor.fetchone()
    avgw2 = avg[0]
    
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Interns';")
    avg = cursor.fetchone()
    avgw3 = avg[0]
    
    cursor.execute("SELECT grade, enter_ctc, CAST(REGEXP_REPLACE(level_, '[^\d]', '', 'g') AS INT) AS levels FROM list;")

    # Fetch all the rows from the result set
    rows = cursor.fetchall()
    
    
    
    cursor.execute("SELECT designation, COUNT(designation) AS Dcount FROM list GROUP BY designation;")
    rows4 = cursor.fetchall()
    
    
    cursor.execute("SELECT COUNT(department) AS Dcount, SUM(enter_ctc) FROM list GROUP BY department;")
    rows5 = cursor.fetchall()
    
    

    # Close the cursor and database connection
    cursor.close()
    conn.close()
    
    data6 = [['Grade', 'Employee CTC', 'Level']]
    data7 = [['Designation', 'Count']]
    data8 = [['Count of Departments', 'CTC']]

    # Append rows from the database to the data6 variable
    for row in rows:
        data6.append(list(row))
    
    for row in rows4:
        data7.append(list(row))
    
    
    for row in rows5:
        data8.append(list(row))


    
    html_code = '''
    <html>
    <head>
      <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
      <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
      <script type="text/javascript">
        google.charts.load("current", {packages:["corechart"]});
        google.charts.setOnLoadCallback(drawCharts);

        function drawCharts() {
          var data4 = google.visualization.arrayToDataTable({{ data6|safe }});

            var options4 = {
              title: 'Grade Levels',
              hAxis: { title: 'Employee CTC' },
              vAxis: { title: 'Level' },
              bubble: {
                textStyle: { fontSize: 11 },
                stroke: '#000000', // Set the stroke color for the bubbles
                fill: '#ff0000', // Set the fill color for the bubbles
                opacity: 0.6, // Set the opacity for the bubbles (0.0 - 1.0)
                sizes: [10, 30], // Set the range of sizes for the bubbles
              },
        };

            var chart4 = new google.visualization.BubbleChart(document.getElementById('chart'));
            chart4.draw(data4, options4);
    }
    

    
          $(document).ready(function() {
          $('#home').click(function() {
            window.location.href = "{{ url_for('home') }}";
          });
          $('#graph1').click(function() {
            window.location.href = "{{ url_for('graph1') }}";
          });
          $('#graph2').click(function() {
            window.location.href = "{{ url_for('graph2') }}";
          });
          $('#graph3').click(function() {
            window.location.href = "{{ url_for('graph3') }}";
          });
          $('#graph4').click(function() {
            window.location.href = "{{ url_for('graph4') }}";
          });
          $('#graph5').click(function() {
            window.location.href = "{{ url_for('graph5') }}";
          });
          $('#graph6').click(function() {
            window.location.href = "{{ url_for('graph6') }}";
          });
        });
      </script>
      <link rel="stylesheet" type="text/css" href="/static/styling.css">
    </head>
    <body>
    <ul class="graph-menu">
        <li><button id="home">Home Page</button></li>
        <li><button id="graph1">Martital Status</button></li>
        <li><button id="graph2">Gender Ratio</button></li>
        <li><button id="graph3">Part Time vs Full time</button></li>
        <li><button id="graph4">Grade vs Level View</button></li>
        <li><button id="graph5">Designation and its Count</button></li>
        <li><button id="graph6">Dept. VS CTC</button></li>
    </ul>
    <div class="container">
        <h1 class="chart-title">Grade vs Level View</h1>
        <div class="charts-container">
            <div id="chart" class="chart"></div> 
        </div>
    </div>
    </body>
    </html>
    '''

    return render_template_string(html_code, data8 = data8, data7 = data7, count1=count1, count2=count2, count1a=count1a, count2a=count2a, avgl1=avgl1, avgl2=avgl2, avgl3=avgl3, avgw1=avgw1, avgw2=avgw2, avgw3=avgw3, data6=data6)



@app.route('/graph5')
def graph5():
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        database="Candidate_List",
        user="postgres",
        password="Adduvaith789",
        host="localhost",
        port="5432"
    )

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # For the Marital Status - Pie chart
    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Married';")
    rows1 = cursor.fetchone()

    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Single';")
    rows2 = cursor.fetchone()

    count1 = rows1[0]
    count2 = rows2[0]

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='male';")
    rows1a = cursor.fetchone()

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='female';")
    rows2a = cursor.fetchone()

    count1a = rows1a[0]
    count2a = rows2a[0]
    
    
    #line chart of Grades
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Permanent';")
    avg = cursor.fetchone()
    avgl1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Contract';")
    avg = cursor.fetchone()
    avgl2 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Consultant';")
    avg = cursor.fetchone()
    avgl3 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Probationary';")
    avg = cursor.fetchone()
    avgw1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Freelancers';")
    avg = cursor.fetchone()
    avgw2 = avg[0]
    
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Interns';")
    avg = cursor.fetchone()
    avgw3 = avg[0]
    
    cursor.execute("SELECT grade, enter_ctc, CAST(REGEXP_REPLACE(level_, '[^\d]', '', 'g') AS INT) AS levels FROM list;")

    # Fetch all the rows from the result set
    rows = cursor.fetchall()
    
    
    
    cursor.execute("SELECT designation, COUNT(designation) AS Dcount FROM list GROUP BY designation;")
    rows4 = cursor.fetchall()
    
    
    cursor.execute("SELECT COUNT(department) AS Dcount, SUM(enter_ctc) FROM list GROUP BY department;")
    rows5 = cursor.fetchall()
    
    

    # Close the cursor and database connection
    cursor.close()
    conn.close()
    
    data6 = [['Grade', 'Employee CTC', 'Level']]
    data7 = [['Designation', 'Count']]
    data8 = [['Count of Departments', 'CTC']]

    # Append rows from the database to the data6 variable
    for row in rows:
        data6.append(list(row))
    
    for row in rows4:
        data7.append(list(row))
    
    
    for row in rows5:
        data8.append(list(row))


    
    html_code = '''
    <html>
    <head>
      <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
      <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
      <script type="text/javascript">
        google.charts.load("current", {packages:["corechart"]});
        google.charts.setOnLoadCallback(drawCharts);

        function drawCharts() {
          var data4 = google.visualization.arrayToDataTable({{ data7|safe }});

          var options4 = {
            title: 'Designation and their Count',
            vAxis: { title: 'Count' },
            legend: { position: 'none' }
          };

          var chart4 = new google.visualization.ColumnChart(document.getElementById('column_chart'));
          chart4.draw(data4, options4);
    }
    

    
          $(document).ready(function() {
          $('#home').click(function() {
            window.location.href = "{{ url_for('home') }}";
          });
          $('#graph1').click(function() {
            window.location.href = "{{ url_for('graph1') }}";
          });
          $('#graph2').click(function() {
            window.location.href = "{{ url_for('graph2') }}";
          });
          $('#graph3').click(function() {
            window.location.href = "{{ url_for('graph3') }}";
          });
          $('#graph4').click(function() {
            window.location.href = "{{ url_for('graph4') }}";
          });
          $('#graph5').click(function() {
            window.location.href = "{{ url_for('graph5') }}";
          });
          $('#graph6').click(function() {
            window.location.href = "{{ url_for('graph6') }}";
          });
        });
      </script>
      <link rel="stylesheet" type="text/css" href="/static/styling.css">
    </head>
    <body>
    <ul class="graph-menu">
        <li><button id="home">Home Page</button></li>
        <li><button id="graph1">Martital Status</button></li>
        <li><button id="graph2">Gender Ratio</button></li>
        <li><button id="graph3">Part Time vs Full time</button></li>
        <li><button id="graph4">Grade vs Level View</button></li>
        <li><button id="graph5">Designation and its Count</button></li>
        <li><button id="graph6">Dept. VS CTC</button></li>
    </ul>
    <div class="container">
        <h1 class="chart-title">Designation and its Count</h1>
        <div class="charts-container">
            <div id="column_chart" class="chart"></div> 
        </div>
    </div>
    </body>
    </html>
    '''

    return render_template_string(html_code, data8 = data8, data7 = data7, count1=count1, count2=count2, count1a=count1a, count2a=count2a, avgl1=avgl1, avgl2=avgl2, avgl3=avgl3, avgw1=avgw1, avgw2=avgw2, avgw3=avgw3, data6=data6)



@app.route('/graph6')
def graph6():
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        database="Candidate_List",
        user="postgres",
        password="Adduvaith789",
        host="localhost",
        port="5432"
    )

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    # For the Marital Status - Pie chart
    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Married';")
    rows1 = cursor.fetchone()

    cursor.execute("SELECT COUNT(Marital_Status) FROM list WHERE Marital_Status='Single';")
    rows2 = cursor.fetchone()

    count1 = rows1[0]
    count2 = rows2[0]

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='male';")
    rows1a = cursor.fetchone()

    cursor.execute("SELECT COUNT(gender) FROM list WHERE gender='female';")
    rows2a = cursor.fetchone()

    count1a = rows1a[0]
    count2a = rows2a[0]
    
    
    #line chart of Grades
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Permanent';")
    avg = cursor.fetchone()
    avgl1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Contract';")
    avg = cursor.fetchone()
    avgl2 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Consultant';")
    avg = cursor.fetchone()
    avgl3 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Probationary';")
    avg = cursor.fetchone()
    avgw1 = avg[0]

    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Freelancers';")
    avg = cursor.fetchone()
    avgw2 = avg[0]
    
    cursor.execute("SELECT COUNT(Employee_Category) FROM list WHERE Employee_Category='Interns';")
    avg = cursor.fetchone()
    avgw3 = avg[0]
    
    cursor.execute("SELECT grade, enter_ctc, CAST(REGEXP_REPLACE(level_, '[^\d]', '', 'g') AS INT) AS levels FROM list;")

    # Fetch all the rows from the result set
    rows = cursor.fetchall()
    
    
    
    cursor.execute("SELECT designation, COUNT(designation) AS Dcount FROM list GROUP BY designation;")
    rows4 = cursor.fetchall()
    
    
    cursor.execute("SELECT COUNT(department) AS Dcount, SUM(enter_ctc) FROM list GROUP BY department;")
    rows5 = cursor.fetchall()
    
    

    # Close the cursor and database connection
    cursor.close()
    conn.close()
    
    data6 = [['Grade', 'Employee CTC', 'Level']]
    data7 = [['Designation', 'Count']]
    data8 = [['Count of Departments', 'CTC']]

    # Append rows from the database to the data6 variable
    for row in rows:
        data6.append(list(row))
    
    for row in rows4:
        data7.append(list(row))
    
    
    for row in rows5:
        data8.append(list(row))


    
    html_code = '''
    <html>
    <head>
      <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
      <script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
      <script type="text/javascript">
        google.charts.load("current", {packages:["corechart"]});
        google.charts.setOnLoadCallback(drawCharts);

        function drawCharts() {
          var data = new google.visualization.DataTable();
          data.addColumn('number', 'X');
          data.addColumn('number', 'Y');

          // Populate the data rows from the fetched data
          var rows = [
    '''

    for row in rows5:
        html_code += "        [{}],\n".format(','.join(str(value) for value in row))

    html_code += '''
    ];

          data.addRows(rows);

          var options = {
              title: 'Dept. Count vs CTCs',
              hAxis: {title: 'Dept'},
              vAxis: {title: 'CTC'},
              legend: 'none'
          };

          var chart = new google.visualization.ScatterChart(document.getElementById('scatter'));
          chart.draw(data, options);
    }
    

    
          $(document).ready(function() {
          $('#home').click(function() {
            window.location.href = "{{ url_for('home') }}";
          });
          $('#graph1').click(function() {
            window.location.href = "{{ url_for('graph1') }}";
          });
          $('#graph2').click(function() {
            window.location.href = "{{ url_for('graph2') }}";
          });
          $('#graph3').click(function() {
            window.location.href = "{{ url_for('graph3') }}";
          });
          $('#graph4').click(function() {
            window.location.href = "{{ url_for('graph4') }}";
          });
          $('#graph5').click(function() {
            window.location.href = "{{ url_for('graph5') }}";
          });
          $('#graph6').click(function() {
            window.location.href = "{{ url_for('graph6') }}";
          });
        });
      </script>
      <link rel="stylesheet" type="text/css" href="/static/styling.css">
    </head>
    <body>
    <ul class="graph-menu">
        <li><button id="home">Home Page</button></li>
        <li><button id="graph1">Martital Status</button></li>
        <li><button id="graph2">Gender Ratio</button></li>
        <li><button id="graph3">Part Time vs Full time</button></li>
        <li><button id="graph4">Grade vs Level View</button></li>
        <li><button id="graph5">Designation and its Count</button></li>
        <li><button id="graph6">Dept. VS CTC</button></li>
    </ul>
    <div class="container">
        <h1 class="chart-title">Dept. VS CTC</h1>
        <div class="charts-container">
            <div id="scatter" class="chart"></div> 
        </div>
    </div>
    </body>
    </html>
    '''

    return render_template_string(html_code, data8 = data8, data7 = data7, count1=count1, count2=count2, count1a=count1a, count2a=count2a, avgl1=avgl1, avgl2=avgl2, avgl3=avgl3, avgw1=avgw1, avgw2=avgw2, avgw3=avgw3, data6=data6)






if __name__ == '__main__':
    app.run()

In [ ]:
from flask import Flask, jsonify
import psycopg2
from decimal import Decimal

app = Flask(__name__)

def convert_to_float(data):
    if isinstance(data, dict):
        return {convert_to_float(key): convert_to_float(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_float(element) for element in data]
    elif isinstance(data, Decimal):
        return str(data)
    return data

def get_data1():
    conn = psycopg2.connect(
        database="Candidate_List",
        user="postgres",
        password="Adduvaith789",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()

    cursor.execute("SELECT COUNT(*) FROM list WHERE Marital_Status='Married';")
    rows1 = cursor.fetchone()
    married_count = rows1[0]

    cursor.execute("SELECT COUNT(*) FROM list WHERE Marital_Status='Single';")
    rows2 = cursor.fetchone()
    single_count = rows2[0]

    cursor.close()
    conn.close()

    data = {
        "Married": married_count,
        "Single": single_count
    }

    return convert_to_float(data)


@app.route('/')
def home():
    data = get_data1()
    return jsonify(data)


if __name__ == '__main__':
    app.run()


In [ ]:
import psycopg2
from decimal import Decimal
import json

def convert_to_float(data):
    if isinstance(data, dict):
        return {convert_to_float(key): convert_to_float(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_float(element) for element in data]
    elif isinstance(data, Decimal):
        return str(data)
    return data

def get_data1():
    conn = psycopg2.connect(
        database="Candidate_List",
        user="postgres",
        password="Adduvaith789",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()

    cursor.execute("SELECT COUNT(*) FROM list WHERE Marital_Status='Married';")
    rows1 = cursor.fetchone()
    married_count = rows1[0]

    cursor.execute("SELECT COUNT(*) FROM list WHERE Marital_Status='Single';")
    rows2 = cursor.fetchone()
    single_count = rows2[0]

    cursor.close()
    conn.close()

    data = {
        "Married": married_count,
        "Single": single_count
    }

    return convert_to_float(data)

data = get_data1()
json_data = json.dumps(data)
print(json_data)


In [ ]:
from flask import Flask, render_template, jsonify
import subprocess

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('indexGall.html')

@app.route('/graph_data')
def graph_data():
    try:
        result = subprocess.run(['python', 'graph1.py'], capture_output=True, text=True)
        json_data = result.stdout.strip()
        return jsonify(json_data)
    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == '__main__':
    app.run()


<h1>Graph 1</h1>

In [ ]:
from flask import Flask, render_template, jsonify
import subprocess

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('indexG1.html')

@app.route('/graph_data')
def graph_data():
    try:
        result = subprocess.run(['jupyter', 'nbconvert', '--to', 'script', 'graph1.ipynb'], capture_output=True, text=True)
        if result.returncode != 0:
            return jsonify({'error': result.stderr})

        result = subprocess.run(['python', 'graph1.py'], capture_output=True, text=True)
        json_data = result.stdout.strip()

        return json_data, 200, {'Content-Type': 'application/json'}

    except Exception as e:
        return jsonify({'error': str(e)})


if __name__ == '__main__':
    app.run()


In [ ]:
from flask import Flask, render_template, jsonify
import subprocess

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('indexG1.html')

@app.route('/graph_data')
def graph_data():
    try:
        result = subprocess.run(['jupyter', 'nbconvert', '--to', 'script', 'graph1.ipynb'], capture_output=True, text=True)
        if result.returncode != 0:
            return jsonify({'error': result.stderr})

        result = subprocess.run(['python', 'graph1.py'], capture_output=True, text=True)
        json_data = result.stdout.strip()

        return json_data, 200, {'Content-Type': 'application/json'}

    except Exception as e:
        return jsonify({'error': str(e)})


if __name__ == '__main__':
    app.run()


In [ ]:
from flask import Flask, render_template, jsonify
import subprocess

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('menu.html')

@app.route('/graph_data')
def graph_data():
    try:
        result = subprocess.run(['jupyter', 'nbconvert', '--to', 'script', 'graph1.ipynb'], capture_output=True, text=True)
        if result.returncode != 0:
            return jsonify({'error': result.stderr}), 500  # Return error with status code 500

        result = subprocess.run(['python', 'graph1.py'], capture_output=True, text=True)
        json_data = result.stdout.strip()

        return json_data, 200, {'Content-Type': 'application/json'}

    except Exception as e:
        return jsonify({'error': str(e)}), 500  # Return error with status code 500


@app.route('/graph1')
def graph1():
    return render_template('indexG1.html')


@app.route('/graph2')
def graph2():
    return render_template('indexG2.html')


@app.route('/graph3')
def graph3():
    return render_template('indexG3.html')


@app.route('/graph4')
def graph4():
    return render_template('indexG4.html')


@app.route('/graph5')
def graph5():
    return render_template('indexG5.html')


if __name__ == '__main__':
    app.run()


In [ ]:
from flask import Flask, render_template, jsonify
import subprocess

app = Flask(__name__)

def get_graph_data(script_name):
    try:
        subprocess.check_output(['jupyter', 'nbconvert', '--to', 'script', f'{script_name}.ipynb'])
        result = subprocess.check_output(['python', f'{script_name}.py'], text=True)
        return result.strip(), 200, {'Content-Type': 'application/json'}
    except subprocess.CalledProcessError as e:
        return jsonify({'error': e.stderr}), 500
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/')
def home():
    return render_template('menu.html')

@app.route('/graph_data1')
def graph_data1():
    return get_graph_data('graph1')

@app.route('/graph_data2')
def graph_data2():
    return get_graph_data('graph2')

@app.route('/graph_data3')
def graph_data3():
    return get_graph_data('graph3')

@app.route('/graph_data4')
def graph_data4():
    return get_graph_data('graph4')

@app.route('/graph_data5')
def graph_data5():
    return get_graph_data('graph5')

@app.route('/graph_data6')
def graph_data6():
    return get_graph_data('graph6')

@app.route('/graph1')
def graph1():
    return render_template('indexG1.html')

@app.route('/graph2')
def graph2():
    return render_template('indexG2.html')

@app.route('/graph3')
def graph3():
    return render_template('indexG3.html')

@app.route('/graph4')
def graph4():
    return render_template('indexG4.html')

@app.route('/graph5')
def graph5():
    return render_template('indexG5.html')

@app.route('/graph6')
def graph6():
    return render_template('indexG6.html')

if __name__ == '__main__':
    app.run()
